# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [ ]:
# import libraries
import pandas as pd
import re
import pickle
from sqlalchemy import create_engine
from nltk import word_tokenize, pos_tag, ne_chunk, WordNetLemmatizer
from sklearn import metrics
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
# import category_encoders as ce

In [ ]:
# load data from database
engine = create_engine('sqlite:///../data/Disaster_Response.db')
# df = pd.read_sql("SELECT * FROM Disaster_Response", engine) # alternative
df = pd.read_sql_table("Disaster_Response", engine)
X = df.loc[:,'message':'genre']
Y = df.loc[:,'related':]
X.shape, Y.shape

### 2. Write a tokenization function to process your text data

In [ ]:
# regex pattern for detecting a url
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

# define tokenizer
def tokenize(text):
    # get list of all urls using regex
    detected_urls = re.findall(url_regex,text)
    
    # replace each url in text string with placeholder
    for url in detected_urls:
        text = text.replace(url,'urlplaceholder')

    # tokenize text
    tokens = word_tokenize(text)
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()

    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

In [ ]:
# test the tokenizer
text=X.original[0]
print(tokenize(text))

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
# simple 3-step pipeline for text-processing and classification
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf',RandomForestClassifier(random_state=0))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=1234)

# train the pipeline
pipeline.fit(X_train.message, y_train)

# predict on test data
y_pred = pipeline.predict(X_test.message)

In [ ]:
y_test.shape, y_pred.shape

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
# show prediction results
for i, col in enumerate(Y.columns):
    print('\033[1m'+col+'\033[0m', '\n', metrics.classification_report(y_test.iloc[:,i], y_pred[:,i]))

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
# for gird-search MultiOutputClassifier is used 
# for extending classifiers that do not natively support multi-target classification 

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=0)))
])

In [ ]:
# peek at tunable parameters
pipeline.get_params()

In [ ]:
# setup tunable parameters
parameters = {
#             'vect__ngram_range': ((1, 1), (1, 2)),
#             'vect__max_df': (0.5, 0.75, 1.0),
#             'vect__max_features': (None, 5000, 10000),
              'tfidf__use_idf': (True, False),
              'clf__estimator__n_estimators': [200, 500],
#               'clf__estimator__min_samples_split': [2,4,8]
              }

In [ ]:
%%time
# setup gridsearch
cv = GridSearchCV(pipeline, param_grid=parameters,return_train_score=True, verbose=5, n_jobs=-1) 
# fit the data
cv.fit(X_train.message, y_train)

In [ ]:
cv.best_score_

In [ ]:
cv.best_params_

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
# predict and show the result
y_pred_tuned = cv.predict(X_test.message)

for i, col in enumerate(Y.columns):
    print('\033[1m'+col+'\033[0m', '\n', metrics.classification_report(y_test.iloc[:,i], y_pred_tuned[:,i]))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
%%time

# add categorical feature through the pipeline
preprocessor = ColumnTransformer(transformers=[
     ('text', TfidfVectorizer(stop_words = 'english', ngram_range=(1,3)), 'message'), # 'message' VS ['message'] why error??
     ('cate', OneHotEncoder(dtype='int', handle_unknown='ignore'), ['genre'])
])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
model = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', RandomForestClassifier(random_state=0,n_estimators=500,n_jobs=-1))])

# fit X_train isntead of X_train.message
model.fit(X_train, y_train)

In [ ]:
# predict and show the result
y_pred_multi_feat_lgb = model.predict(X_test)

for i, col in enumerate(Y.columns):
    print('\033[1m'+col+'\033[0m', '\n', metrics.classification_report(y_test.iloc[:,i], y_pred_multi_feat_lgb[:,i]))

### 9. Export your model as a pickle file

In [ ]:
with open('../models/model.pkl', 'wb') as file:
    pickle.dump(model, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.